In [5]:
#Matthew Lund
#mtlund@wpi.edu
#ECE577 Homework3

#imported libs

#For grabbing ImageNet Images and picking 10 Random images
import os
import random

#Tensorflow and Keras
import tensorflow as tf
from tensorflow import keras
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.resnet50 import preprocess_input

#Numpy and Plotting
import numpy as np
import matplotlib.pyplot as plt
#Using Jupyter Notebook
%matplotlib inline

#Foolbox
import foolbox
from foolbox.attacks import BlendedUniformNoiseAttack, FGSM, ContrastReductionAttack, SinglePixelAttack, SaliencyMapAttack


In [6]:
model = ResNet50(weights='imagenet', include_top=True)
preprocessing = (np.array([103.939, 116.779, 123.68]), 1)
fmodel = foolbox.models.KerasModel(model, bounds=(0, 255), 
                                   preprocessing=preprocessing)

In [7]:
#Foolbox attacks used
attacks = [BlendedUniformNoiseAttack(fmodel), 
          ContrastReductionAttack(fmodel), 
          FGSM(fmodel), 
          SinglePixelAttack(fmodel), 
          SaliencyMapAttack(fmodel)]


In [8]:
# Path to images folder using ILSVRC2012
image_folder_path = './Images'

# get a list of all jpeg files in the folder
all_images = [f for f in os.listdir(image_folder_path) if f.endswith('.jpeg')]

# pick 10 images at random
selected_images = random.sample(all_images, 10)

# load the images using keras
loaded_images = [load_img(os.path.join(image_folder_path, img), target_size=(224, 224)) for img in selected_images]

# convert the images to numpy arrays and preprocess them for ResNet50
preprocessed_images = [preprocess_input(img_to_array(img)) for img in loaded_images]

ValueError: Sample larger than population or is negative

In [ ]:
#For Loop for Images
for img in preprocessed_images:
    # convert the image to a batch of size 1
    img_batch = np.expand_dims(img, axis=0)
    
    # get the label of the image
    label = np.argmax(model.predict(img_batch))
    
    for attack in attacks:
        # apply the attack
        adversarial = attack(img_batch, label)
        
        # calculate the noise
        noise = adversarial - img_batch
        
        # get the name of the attack
        attack_name = attack.__class__.__name__
        
        # plot the original, adversarial, and noise images
        plt.figure(figsize=(10, 10))
        
        plt.subplot(1, 3, 1)
        plt.imshow(img / 255)  # scale the values to [0, 1] for plotting
        plt.title('Original')
        
        plt.subplot(1, 3, 2)
        plt.imshow(adversarial[0] / 255)  # scale the values to [0, 1] for plotting
        plt.title(f'Adversarial ({attack_name})')
        
        plt.subplot(1, 3, 3)
        plt.imshow(noise[0] / 255)  # scale the values to [0, 1] for plotting
        plt.title('Noise')
        
        plt.show()